In [3]:
import glob, json

import sys
sys.path.append(r"D:\ig_pipeline")

from b1k_pipeline.utils import parse_name, get_targets, PIPELINE_ROOT

files = []
for fn in [PIPELINE_ROOT / "cad" / tgt / "artifacts/collision_selection.json" for tgt in get_targets("combined")]:
    with open(fn) as f:
        files.append((fn, json.load(f)))

In [4]:
len(files)

81

In [4]:
canonical_cats = {}
base_link_shows_w_name = {}
for fn in [PIPELINE_ROOT / "cad" / tgt / "artifacts/object_list.json" for tgt in get_targets("combined")]:
    with open(fn) as f:
        data = json.load(f)
        for obj in data["provided_objects"]:
            cat, model = obj.split("-")
            if model in canonical_cats:
                print(f"{fn} {obj} previously seen as {canonical_cats[model]}")
            canonical_cats[model] = cat
            
        for mn in data["meshes"]:
            m = parse_name(mn)
            if m.group("bad"):
                continue
            if int(m.group("instance_id")) != 0:
                continue
            model = m.group("model_id")
            if m.group("link_name") in (None, "", "base_link"):
                shows_w_name = m.group("link_name") == "base_link"
                assert model not in base_link_shows_w_name, model
                base_link_shows_w_name[model] = shows_w_name
                
        canonical_cat_keys = set(canonical_cats.keys())
        blswn_keys = set(base_link_shows_w_name.keys())
        assert canonical_cat_keys == blswn_keys, [f"{canonical_cats[k]}-{k}" for k in canonical_cat_keys - blswn_keys]

In [5]:
import collections
for fn, all_selections in files:
    print()
    grouped_selections = collections.defaultdict(set)
    for k, v in all_selections.items():
        m = parse_name(k)
        cat = m["category"]
        model = m["model_id"]
        link = m["link_name"]
        if model not in canonical_cats:
            continue
        if (link in (None, "") and base_link_shows_w_name[model]) or (link == "base_link" and not base_link_shows_w_name[model]):
            continue
        if not link:
            link = "base_link"
        grouped_selections[(model, link)].add((k, v))
        
    # Remove non-canonical if canonical exists
    new_grouped_selections = {}
    for (model, link), selections in grouped_selections.items():
        has_canonical = any(parse_name(k)["category"] == canonical_cats[model] for k, v in selections)
        if has_canonical:
            selections = [(k, v) for k, v in selections if parse_name(k)["category"] == canonical_cats[model]]
        new_grouped_selections[(model, link)] = selections
        
    problems = [(k, v) for k,v in new_grouped_selections.items() if len(v)>1]
    if problems:
        print(f"In {fn}: {problems}")
    else:
        with open(fn, "w") as f:
            json.dump({k: v for selections in new_grouped_selections.values() for k, v in selections}, f, sort_keys=True, indent=4)

In [5]:
to_remove = [
"zppyty",
"gaqzfq",
"kqokhv",
"oddyhd",
"kmgltg",
"dxwbae",
"dxwbae",
"xjyvwx",
"cdjsgw",
"ieyjnv",
"saldxe",
"xrozyp",
"ycspus",
"rnkvca",
"iueiur",
]

import collections
for fn, all_selections in files:
    new_selections = {x: y for x, y in all_selections.items() if parse_name(x).group("model_id") not in to_remove}
    with open(fn, "w") as f:
        json.dump({k: v for selections in new_grouped_selections.values() for k, v in selections}, f, sort_keys=True, indent=4)

NameError: name 'new_grouped_selections' is not defined